In [ ]:
# !pip install tqdm

In [1]:
import pandas as pd

In [2]:
aggregated = pd.read_csv('../data/marks_csv/aggregated.csv', index_col=None)

In [3]:
aggregated.head()

,input,output,confidence
0,Условия термодинамической согласованности имею...,NEUTRAL,0.92
1,"мишени, подтвержденная там для 2D  модели, до...",NEUTRAL,0.90
2,При разработке алгоритмов решения задач ГД и М...,NEUTRAL,0.97
3,На основе теории стохастических динамических п...,GOOD,1.00
4,Основным достоинством лазерного поджига являет...,NEUTRAL,0.61


In [4]:
aggregated['input'] = [' '.join(line.split()) for line in aggregated['input']]

In [5]:
def get_index_start(line: str):
    lower_line = line.lower()
    if 'введение' in lower_line:
        res = lower_line.index('введение') + len('введение')
        return res
    else:
        res = len(line) - ''.join(list(reversed(line))).index('...')
        return res
lines = aggregated['input']
lines = [l[get_index_start(l):] if '.....' in l or '. . . .' in l else l for l in lines]
lines = [l.strip() for l in lines]
aggregated['input'] = lines
aggregated.reset_index(drop=True, inplace=True)

In [6]:
copyes = []

lines = aggregated['input']
output = aggregated['output']

for i1, line1 in enumerate(lines):
    # print(i1 / aggregated.shape[0])
    for i2, line2 in enumerate(lines[i1+1:]):
        if i2 <= i1:
            continue
#         line1 = row1['input']
#         line2 = row2['input']

        if line1[:150] ==  line2[:150]:
            copyes.append((i1, i1+i2+1))
len(copyes), len([1 for i1, i2 in copyes if output[i1] != output[i2]])

(792, 335)

In [7]:
del_start = set(c[1] for c in copyes)
aggregated['del_start'] = [i in del_start for i in range(aggregated.shape[0])]
aggregated = aggregated[~aggregated.del_start]
aggregated.reset_index(drop=True, inplace=True)

In [8]:
copyes = []

lines = aggregated['input']
output = aggregated['output']

for i1, line1 in enumerate(lines):
    # print(i1 / aggregated.shape[0])
    for i2, line2 in enumerate(lines[i1+1:]):
        if i2 <= i1:
            continue
#         line1 = row1['input']
#         line2 = row2['input']

        if line1[150:] ==  line2[150:]:
            copyes.append((i1, i1+i2+1))
len(copyes), len([1 for i1, i2 in copyes if output[i1] != output[i2]])

(3, 2)

In [15]:
del_finish = set(c[1] for c in copyes)
aggregated['del_finish'] = [i in del_finish for i in range(aggregated.shape[0])]
aggregated = aggregated[~aggregated.del_finish]
aggregated.reset_index(drop=True, inplace=True)

In [18]:
aggregated[['input', 'output', 'confidence']].to_csv('../data/marks_csv/aggregated_clear.csv', index=None)